In [0]:
%pip install --upgrade "mlflow[databricks]>=3.1.0" openai "databricks-connect>=16.1" "databricks-langchain>=0.3.0"
dbutils.library.restartPython()

### Reference:
https://learn.microsoft.com/en-ca/azure/databricks/mlflow3/genai/tracing/app-instrumentation/automatic

Notice that we get the trace without an associated run (navigate to experiment)


In [0]:
import mlflow
from databricks.sdk import WorkspaceClient
from openai import OpenAI

# Enable MLflow's autologging to instrument your application with Tracing
mlflow.openai.autolog()

# Create an OpenAI client that is connected to Databricks hosted LLMs
mlflow_creds = mlflow.utils.databricks_utils.get_databricks_host_creds()
client = OpenAI(
    api_key=mlflow_creds.token,
    base_url=f"{mlflow_creds.host}/serving-endpoints"
)

# Use the trace decorator to capture the application's entry point
@mlflow.trace
def my_app(input: str):
  # This call is automatically instrumented by `mlflow.openai.autolog()`
  response = client.chat.completions.create(
    model="databricks-meta-llama-3-1-8b-instruct",  # This example uses a Databricks hosted LLM - you can replace this with any AI Gateway or Model Serving endpoint. If you have an external model endpoint configured, you can use that model name here.
    messages=[
      {
        "role": "system",
        "content": "You are a helpful assistant.",
      },
      {
        "role": "user",
        "content": input,
      },
    ],
  )
  return response.choices[0].message.content

result = my_app(input="What is MLflow?")
print(result)

## Note that you are not required to use open ai client

and notice that the trace is stored with experiment without differentiation.

In [0]:
from databricks_langchain import ChatDatabricks
import mlflow


chat_model = ChatDatabricks(
    endpoint="databricks-meta-llama-3-1-8b-instruct"
)

@mlflow.trace
def get_chat_response(messages):
  result = chat_model.invoke(messages)
  return result

messages=[
      {
        "role": "system",
        "content": "You are a helpful assistant.",
      },
      {
        "role": "user",
        "content": "what is mlflow",
      },
    ]
  

#result = chat_model.invoke(messages)
result = get_chat_response(messages)
print(result.content)
